In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

dbconn = os.getenv("DBCONN")
dbconn


In [ ]:
#  %pip install psycopg2

import psycopg2

# establish database connection using connection string from .env
conn = psycopg2.connect(dbconn)

# check if the connection is open
# if conn.closed:
#     raise Exception("Connection to the database failed")
# # print connection status
# print("Connection to the database is open")
# # print connection info
# print(conn.get_dsn_parameters())
# # print server version
# print("Server version:", conn.server_version)


# create a cursor
cur = conn.cursor()

In [ ]:
cur.execute('''
        CREATE TABLE IF NOT EXISTS daily_prices (
            Date TIMESTAMP,
            Open FLOAT,
            High FLOAT,
            Low FLOAT,
            Close FLOAT,
            Volume INTEGER,
            PRIMARY KEY(Date)
        )
    ''')

In [ ]:
conn.commit()

In [ ]:
# close the cursor and sever connection
cur.close()
conn.close()

In [ ]:
api_key = os.getenv("APLHAVANTAGEAPIKEY")

print(api_key)

In [ ]:
import requests

url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=" + api_key
response = requests.get(url)
print(response.status_code)
data = response.json()

# Display the latest 5 days of daily prices in a readable format
time_series = data.get("Time Series (Daily)", {})
for date in sorted(time_series.keys(), reverse=True)[:5]:
    print(f"Date: {date}")
    for key, value in time_series[date].items():
        print(f"  {key}: {value}")
    print("-" * 30)

In [ ]:
import psycopg2

def insert_data_pg(date, open_price, high_price, low_price, close_price, volume):
    conn = psycopg2.connect(dbconn)
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO daily_prices (Date, Open, High, Low, Close, Volume)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (Date) DO UPDATE SET
            Open = EXCLUDED.Open,
            High = EXCLUDED.High,
            Low = EXCLUDED.Low,
            Close = EXCLUDED.Close,
            Volume = EXCLUDED.Volume
    ''', (date, open_price, high_price, low_price, close_price, volume))
    conn.commit()
    cursor.close()
    conn.close()

# Insert the data into the database
for date, prices in time_series.items():
    insert_data_pg(
        date,
        float(prices["1. open"]),
        float(prices["2. high"]),
        float(prices["3. low"]),
        float(prices["4. close"]),
        int(prices["5. volume"])
    )

def query_data_pg():
    conn = psycopg2.connect(dbconn)
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM daily_prices ORDER BY Date DESC LIMIT 5')
    rows = cursor.fetchall()
    cursor.close()
    conn.close()
    return rows

# Print the last 5 entries from the database
last_entries = query_data_pg()
for entry in last_entries:
    print(f"Date: {entry[0]}, Open: {entry[1]}, High: {entry[2]}, Low: {entry[3]}, Close: {entry[4]}, Volume: {entry[5]}")




In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = "https://u.today/search/node?keys=bitcoin"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract article titles, dates, authors, and links
main_block = soup.find('main', class_='main-block')
articles = main_block.find_all('div', class_='news__item') if main_block else []
data = []
seen = set()
for article in articles:
    # Title
    title_tag = article.find('div', class_='news__item-title')
    title = title_tag.get_text(strip=True) if title_tag else 'No Title'
    # Date
    date_tag = article.find('div', class_='humble')
    date = date_tag.get_text(strip=True) if date_tag else 'No Date'
    # Author
    author_tag = article.find('a', class_='humble humble--author')
    author = author_tag.get_text(strip=True) if author_tag else 'Unknown'
    # Link
    link_tag = article.find('a', class_='news__item-body')
    link = link_tag['href'] if link_tag and link_tag.has_attr('href') else 'No Link'

    # Use (title, date, author, link) as a unique identifier
    identifier = (title, date, author, link)
    if identifier not in seen:
        data.append({
            'Title': title,
            'Date': date,
            'Author': author,
            'Link': link
        })
        seen.add(identifier)

# Create a DataFrame from the scraped data
df = pd.DataFrame(data)
df

In [ ]:
# import data from df into PostgreSQL database
def insert_news_data_pg(title, date, author, link):
    conn = psycopg2.connect(dbconn)
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO news (Title, Date, Author, Link)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (Title, Date) DO NOTHING
    ''', (title, date, author, link))
    conn.commit()
    cursor.close()
    conn.close()
# Create the news table if it doesn't exist
def create_news_table():
    conn = psycopg2.connect(dbconn)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS news (
            Title TEXT,
            Date TEXT,
            Author TEXT,
            Link TEXT,
            PRIMARY KEY (Title, Date)
        )
    ''')
    conn.commit()
    cursor.close()
    conn.close()
create_news_table()
# Insert the data into the database
for index, row in df.iterrows():
    insert_news_data_pg(row['Title'], row['Date'], row['Author'], row['Link'])
# Function to query the news data from PostgreSQL
def query_news_data_pg():
    conn = psycopg2.connect(dbconn)
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM news ORDER BY Date DESC LIMIT 5')
    rows = cursor.fetchall()
    cursor.close()
    conn.close()
    return rows
# Print the last 5 news entries from the database
last_news_entries = query_news_data_pg()
for entry in last_news_entries:
    print(f"Title: {entry[0]}, Date: {entry[1]}, Author: {entry[2]}, Link: {entry[3]}")
    


In [ ]:
import streamlit as st

st.title("Hello World")
